In [1]:
#!/usr/bin/env python
import re, sys, math
import glob
import random
import json
import os 
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import log_loss
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
from collections import OrderedDict
import pandas as pd
from helper_utilities import *
from helper_plotutilities import *
from unet_model  import *
from unet_predict_sk import *

%matplotlib inline

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.



Successfully imported packages!!!

START: 1526623808.7187593

Successfully imported packages!!!



In [7]:
##explist256 = ['combined_1_3_256_epoch20', 'combined_1_3_256', 'combined_aug_1_3_256',\
# 'combined_drop_1_3_256','combined_aug_drop_1_3_256']
keys = ["samples", "logloss", "weighted_logloss","accuracy", "weighted_accuracy", "precision","recall", "f1_score", "true_positive", \
       "false_positive","true_negative","false_negative", "zero_contour_labels", "zero_contour_pred", \
       "missed_pred_lt_05", "missed_pred_gt_25", "missed_pred_gt_50", "missed_pred_eq_100"]

explist256 = [ 'combined_1_3_256', 'combined_aug_1_3_256', 'combined_drop_1_3_256','combined_aug_drop_1_3_256']

#image set that includes labels with zero contours in the training set
#explist256_0 = ['combined_1_3_0_256', 'combined_1_3_0_256_dice','combined_1_3_0_256_aug_dice',  'combined_aug_1_3_0_256', 'combined_1_3_0_256_drop','combined_1_3_0_256_dicecoef', 'combined_1_3_0_256_drop_dicecoef']
explist256_0 = [ 'combined_1_3_0_256_dice','combined_1_3_0_256_aug_dice']

#explist176 = ['combined_1_3_176', 'combined_1_3_176_aug_dice', 'combined_1_3_176_aug_drop_dice']
explist176 = ['combined_1_3_176', 'combined_1_3_176_dice', 'combined_1_3_176_aug_dice', 'combined_1_3_176_drop_dice', 'combined_1_3_176_aug_drop_dice']

explist176_0 = ['combined_1_3_0_176_dice', 'combined_1_3_0_176_aug_dice','combined_1_3_0_176_drop_dice', 'combined_1_3_0_176_aug_drop_dice', 'combined_1_3_0_176_aug_drop_dice2']
explist176_0_bn = ['1_3_0_176_bn', '1_3_0_176_bn2', '1_3_0_176_bn4','1_3_0_176_bn_drop', '1_3_0_176_bn_aug_drop']
explist256_0_bn = ['1_3_0_256_bn_dice']

bd176 = ['1_3_0_176_aug_bd', '1_3_0_176_aug_bd2']


In [3]:

            
#################
# Method to create a U-net model and train it
# Create a U-Net model, train the model and run the predictions and save the trained weights and predictions
#
##########################
# /masvol/heartsmart/unet_model/data/sunnybrook_176_train_images.npy 
# /masvol/heartsmart/unet_model/data/sunnybrook_176_train_images.npy 
# /masvol/heartsmart/unet_model/data/sunnybrook_176_train_labels.npy

def train_unet_model (model_name, nGPU, image_size, training_images, training_labels, test_images, test_labels, model_path, dropout, optimizer, learningrate, lossfun, batch_size, epochs, augmentation = False, model_summary = False):
    
    #samples, x, y, z = pred.shape
    
    train_data = {}
    test_data = {}

    train_data["images"] = training_images
    train_data["labels"] = training_labels
    test_data["images"] = test_images
    test_data["labels"] = test_labels

    
    if not os.path.exists(model_path):
        print ("creating dir ", model_path)
        os.makedirs(model_path)
            
    # get the u-net model and load train and test data
    myunet = myUnet(model_name = model_name, nGPU = nGPU, image_size = image_size, dropout = dropout, optimizer = optimizer, lr=learningrate, loss_fn = lossfn)
    myunet.load_data (train_data, test_data)

    if (model_summary == True):
        print ("Printing model summary ")
        #myunet.model.summary()
        myunet.parallel_model.summary()
        
    res = myunet.train_and_predict(model_path, batch_size = batch_size, nb_epoch = epochs, augmentation = augmentation)
    
#     if (augmentation == True) :
#         res = myunet.train_with_augmentation(model_file, batch_size = batch_size, nb_epoch = epochs)
#     else :
#         res = myunet.train_and_predict(model_file, batch_size = batch_size, nb_epoch = epochs)
        
    return myunet

def predict_with_pretrained_weights(model_name, nGPU, model_file, image_size, test_image_file, test_label_file = "none"):
    img_size_list = [176, 256]
    if image_size not in img_size_list:
        print ("image size %d is not supported"%image_size)
        return 
    
    size_str = str(image_size)
    
    test_data = {}
    test_data["images"] = test_image_file
    test_data["labels"] = test_label_file

    model_file = model_file

    print('-'*30)
    print ("Get Test images and labels...")
    if test_label_file == "none": 
        ts = load_images(test_image_file)
        tl = "none"
    else :
        ts , tl= load_images_and_labels(test_data)

    print('-'*30)
    print ("Creating U-net model...")
    myunet = myUnet(model_name = model_name, nGPU = nGPU, image_size = image_size)
    print('-'*30)
    print ("Loading the pre-trained weights...")
    myunet.load_pretrained_weights(model_file)
    print('-'*30)

    print('Run predictions...')
    myunet.predict(test_image_array = ts, test_label_array = tl)
    print('-'*30)

    return myunet

In [ ]:
len(threshold), fpr.max(),

In [8]:
# RESULTS_PATH = '/masvol/heartsmart/unet_model/baseline/results/'
# image_file = "/masvol/heartsmart/unet_model/baseline/data/combined_1_3_0_176_test_images.npy"
# #label_file = "/masvol/heartsmart/unet_model/baseline/data/combined_1_3_0_176_test_labels.npy"

# for results_file in bd176 :
#     m_name = results_file
#     pred_file = RESULTS_PATH + results_file + 'z_predictions.npy'
#     weights_file = RESULTS_PATH + results_file + '.hdf5'
#     print ("weights file", weights_file)
#     print ("pred_file", pred_file)


#     mnet = predict_with_pretrained_weights(model_name=m_name, nGPU=0, model_file=weights_file, image_size = 176, \
#                                 test_image_file = image_file)
#     #mnet.save_model_info(mpath)
#     print ("Saving predictions")
#     np.save(pred_file, mnet.predictions)

weights file /masvol/heartsmart/unet_model/baseline/results/1_3_0_176_aug_bd.hdf5
pred_file /masvol/heartsmart/unet_model/baseline/results/1_3_0_176_aug_bdz_predictions.npy
------------------------------
Get Test images and labels...
------------------------------
load np arrays of images ...
------------------------------
Loading files :  /masvol/heartsmart/unet_model/baseline/data/combined_1_3_0_176_test_images.npy
shape, max, min, mean of original image set: (270, 176, 176, 1) 1.0 0.0 0.20691323
shape, max, min, mean after normalization: (270, 176, 176, 1) 1.0 0.0 0.20691323
------------------------------
Creating U-net model...
------------------------------
Building U-net model
------------------------------
conv1 shape: (?, 176, 176, 64)
conv1 shape: (?, 176, 176, 64)
pool1 shape: (?, 88, 88, 64)
conv2 shape: (?, 88, 88, 128)
conv2 shape: (?, 88, 88, 128)
pool2 shape: (?, 44, 44, 128)
conv3 shape: (?, 44, 44, 256)
conv3 shape: (?, 44, 44, 256)
pool3 shape: (?, 22, 22, 256)
Adding

/masvol/heartsmart/unet_model/unet_model.py:193: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("co...)`
  self.model = Model(input = inputs, output = conv10)


------------------------------
------------------------------
Run predictions...
------------------------------
predict test data....
270/270 [==============================] - 3s 11ms/step
------------------------------
------------------------------
------------------------------
Saving predictions
weights file /masvol/heartsmart/unet_model/baseline/results/1_3_0_176_aug_bd2.hdf5
pred_file /masvol/heartsmart/unet_model/baseline/results/1_3_0_176_aug_bd2z_predictions.npy
------------------------------
Get Test images and labels...
------------------------------
load np arrays of images ...
------------------------------
Loading files :  /masvol/heartsmart/unet_model/baseline/data/combined_1_3_0_176_test_images.npy
shape, max, min, mean of original image set: (270, 176, 176, 1) 1.0 0.0 0.20691323
shape, max, min, mean after normalization: (270, 176, 176, 1) 1.0 0.0 0.20691323
------------------------------
Creating U-net model...
------------------------------
Building U-net model
----

In [9]:
# RESULTS_PATH = '/masvol/heartsmart/unet_model/baseline/results/'
# label_file176 = '/masvol/heartsmart/unet_model/baseline/data/combined_1_3_0_176_test_labels.npy'

# perf176bd = pd.DataFrame( keys, columns=["Attribute"] )
# for results_file in bd176 :
#     pred_file = RESULTS_PATH + results_file + '_predictions.npy'
#     perf = compute_performance_statistics(label_file176, pred_file)
#     perf176bd[results_file]= perf176bd['Attribute'].map(perf)

#     results_filep = results_file+"z"
#     pred_file = RESULTS_PATH + results_filep + '_predictions.npy'
#     perf = compute_performance_statistics(label_file176, pred_file)
#     perf176bd[results_filep]= perf176bd['Attribute'].map(perf)


In [10]:
perf176bd

,Attribute,1_3_0_176_aug_bd,1_3_0_176_aug_bdz,1_3_0_176_aug_bd2,1_3_0_176_aug_bd2z
0,samples,2.700000e+02,2.700000e+02,2.700000e+02,2.700000e+02
1,logloss,3.314687e-02,3.314687e-02,3.286729e-02,3.286729e-02
2,weighted_logloss,7.729452e-02,7.729452e-02,8.872644e-02,8.872644e-02
3,accuracy,9.673965e-01,9.673965e-01,9.676670e-01,9.676670e-01
4,weighted_accuracy,9.256172e-01,9.256172e-01,9.150959e-01,9.150959e-01
5,precision,9.452434e-01,9.452434e-01,9.561431e-01,9.561431e-01
6,recall,9.512370e-01,9.512370e-01,9.475576e-01,9.475576e-01
7,f1_score,9.482307e-01,9.482307e-01,9.518310e-01,9.518310e-01
8,true_positive,4.167550e+05,4.167550e+05,4.151430e+05,4.151430e+05
9,false_positive,2.414200e+04,2.414200e+04,1.904200e+04,1.904200e+04


In [ ]:
RESULTS_PATH = '/masvol/heartsmart/unet_model/baseline/results/'
label_file = '/masvol/heartsmart/unet_model/baseline/data/combined_1_3_0_256_test_labels.npy'

perf256_0_df = pd.DataFrame( keys, columns=["Attribute"] )
for results_file in explist256_0 :
    pred_file = RESULTS_PATH + results_file + '_predictions.npy'
    perf = compute_performance_statistics(label_file, pred_file)
    perf256_0_df[results_file]= perf256_0_df['Attribute'].map(perf)
    
    results_filep = results_file+"z"
    pred_file = RESULTS_PATH + results_filep + '_predictions.npy'
    perf = compute_performance_statistics(label_file, pred_file)
    perf256_0_df[results_filep]= perf256_0_df['Attribute'].map(perf)


In [ ]:
perf256_0_df

In [ ]:
#image set that includes labels with zero contours in the training set
col256_0 = ['combined_1_3_0_256', 'combined_aug_1_3_0_256', 'combined_1_3_0_256_drop','combined_1_3_0_256_dicecoef', 'combined_1_3_0_256_drop_dicecoef']

#explist176 = ['combined_1_3_176', 'combined_1_3_176_aug_dice', 'combined_1_3_176_aug_drop_dice']
col176 = ["Attribute",'combined_1_3_176z', 'combined_1_3_176_dicez', 'combined_1_3_176_aug_dicez', 'combined_1_3_176_drop_dicez', 'combined_1_3_176_aug_drop_dicez']

col176_0 = ['combined_1_3_0_176_dice', 'combined_1_3_0_176_aug_dice','combined_1_3_0_176_drop_dice', 'combined_1_3_0_176_aug_drop_dice', 'combined_1_3_0_176_aug_drop_dice2']

perf176_df2 = perf176_df[col176]
perf176_df2

In [ ]:
perf176_df2

In [ ]:
!ls /masvol/heartsmart/unet_model/baseline/results/*.csv

In [ ]:
df1 = pd.read_csv('/masvol/heartsmart/unet_model/baseline/results/combined_1_3_0_176_perf_comparison_old.csv', sep='\t')

In [ ]:
df1

In [ ]:
df2 = pd.read_csv('/masvol/heartsmart/unet_model/baseline/results/1_3_0_176_perf_comparison.csv', sep='\t')

In [ ]:
df2

In [ ]:
file_name = '/masvol/heartsmart/unet_model/baseline/results/1_3_0_256_perf_comparision2.csv'
perf256_0_df.to_csv(file_name, sep='\t')

In [ ]:
label_file = "/masvol/heartsmart/unet_model/baseline/data/combined_1_3_0_176_test_labels.npy"
pred_file = "/masvol/heartsmart/unet_model/baseline/results/combined_1_3_0_176_aug_dice_predictions.npy"
fpr, tpr, threshold, roc_auc = compute_roc_auc(label_file, pred_file)

# method I: plt
import matplotlib.pyplot as plt
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

In [ ]:
#'1_3_0_176_bn_drop_learning_history.json'
# weights_file = "/masvol/heartsmart/unet_model/baseline/results/1_3_0_176_bn2.hdf5"
# #history_file = '/masvol/heartsmart/unet_model/baseline/results/1_3_0_176_bn2_learning_history.json'
# image_file = "/masvol/heartsmart/unet_model/baseline/data/combined_1_3_0_176_test_images.npy"
# label_file = "/masvol/heartsmart/unet_model/baseline/data/combined_1_3_0_176_test_labels.npy"
# pred_file = "/masvol/heartsmart/unet_model/baseline/results/bn_predictions.npy"

weights_file = "/masvol/heartsmart/unet_model/baseline/results/1_3_0_256_bn_dice.hdf5"
history_file = '/masvol/heartsmart/unet_model/baseline/results/1_3_0_256_bn_dice_learning_history.json'
image_file = "/masvol/heartsmart/unet_model/baseline/data/combined_1_3_0_256_test_images.npy"
label_file = "/masvol/heartsmart/unet_model/baseline/data/combined_1_3_0_256_test_labels.npy"
pred_file = "/masvol/heartsmart/unet_model/baseline/results/1_3_0_256_bn_dice_predictions.npy"

m_name = "1_3_0_256_bn_dicep"
mpath = "/masvol/heartsmart/unet_model/baseline/results/"

mnet = predict_with_pretrained_weights(model_name=m_name, nGPU=0, model_file=weights_file, image_size = 256, \
                                test_image_file = image_file, test_label_file = label_file)
#mnet.save_model_info(mpath)

pred_file =  mpath + m_name + "_predictions.npy"
print ("Saving predictions", pred_file)
np.save(pred_file, mnet.predictions)

# plot_learning_history(history_file)
# show_performance_statistics(label_file, pred_file)
# find_outliers(label_file, pred_file)
# display_images_labels_predictions (image_file, label_file, pred_file, num_images = 4, random_images = True)
# print('-'*30)
# print('-'*30)
# zcontour = [4, 8, 32, 39, 63, 65, 67, 77, 83, 104, 144, 169, 198, 201]
# display_images_labels_predictions (image_file, label_file, pred_file, image_list = zcontour)
#combined_1_3_0_176_drop_dice

In [ ]:
!ls -lprt 1_3_0_256_bn_dicep*

In [ ]:
weights_file = "/masvol/heartsmart/unet_model/baseline/results/1_3_0_176_bn2.hdf5"
#history_file = '/masvol/heartsmart/unet_model/baseline/results/1_3_0_176_bn2_learning_history.json'
image_file = "/masvol/heartsmart/unet_model/baseline/data/combined_1_3_0_176_test_images.npy"
label_file = "/masvol/heartsmart/unet_model/baseline/data/combined_1_3_0_176_test_labels.npy"
pred_file = "/masvol/heartsmart/unet_model/baseline/results/bn_predictions.npy"

m_name = "bn"
mpath = "/masvol/heartsmart/unet_model/baseline/results/"


#plot_learning_history(history_file)
show_performance_statistics(label_file, pred_file)
find_outliers(label_file, pred_file)
display_images_labels_predictions (image_file, label_file, pred_file, num_images = 4, random_images = True)
print('-'*30)
print('-'*30)
zcontour = [4, 8, 32, 39, 63, 65, 67, 77, 83, 104, 144, 169, 198, 201]
display_images_labels_predictions (image_file, label_file, pred_file, image_list = zcontour)
#combined_1_3_0_176_drop_dice

In [ ]:
# weights_file = "/masvol/heartsmart/unet_model/baseline/results/combined_1_3_0_176_dice.hdf5"
# history_file = '/masvol/heartsmart/unet_model/baseline/results/combined_1_3_0_176_dice_learning_history.json'
# image_file = "/masvol/heartsmart/unet_model/baseline/data/combined_1_3_0_176_test_images.npy"
# label_file = "/masvol/heartsmart/unet_model/baseline/data/combined_1_3_0_176_test_labels.npy"
# pred_file = "/masvol/heartsmart/unet_model/baseline/results/combined_1_3_0_176_dice_predictions.npy"

# m_name = "unet_prediction"

# mnet = predict_with_pretrained_weights(model_name=m_name, nGPU=0, model_file=weights_file, image_size = 176, \
#                                 test_image_file = image_file, test_label_file = label_file)

In [ ]:

weights_file = "/masvol/heartsmart/unet_model/baseline/results/combined_1_3_0_176_dice.hdf5"
history_file = '/masvol/heartsmart/unet_model/baseline/results/combined_1_3_0_176_dice_learning_history.json'
image_file = "/masvol/heartsmart/unet_model/baseline/data/combined_1_3_0_176_test_images.npy"
label_file = "/masvol/heartsmart/unet_model/baseline/data/combined_1_3_0_176_test_labels.npy"
pred_file = "/masvol/heartsmart/unet_model/baseline/results/combined_1_3_0_176_dice_predictions.npy"
plot_learning_history(history_file)
show_performance_statistics(label_file, pred_file)
find_outliers(label_file, pred_file)
display_images_labels_predictions (image_file, label_file, pred_file, num_images = 4, random_images = True)
print('-'*30)
print('-'*30)
zcontour = [47, 68, 81, 82, 141, 155, 173, 202, 213]
display_images_labels_predictions (image_file, label_file, pred_file, image_list = zcontour)

In [ ]:
RESULTS_PATH = '/masvol/heartsmart/unet_model/baseline/results/'
label_file176_0 = '/masvol/heartsmart/unet_model/baseline/data/combined_1_3_0_176_test_labels.npy'

perf176_0_df = pd.DataFrame( keys, columns=["Attribute"] )
for results_file in explist176_0 :
    pred_file = RESULTS_PATH + results_file + '_predictions.npy'
    perf = compute_performance_statistics(label_file176_0, pred_file)
    perf176_0_df[results_file]= perf176_0_df['Attribute'].map(perf)
    

In [ ]:
perf176_0_df

In [ ]:
RESULTS_PATH = '/masvol/heartsmart/unet_model/baseline/results/'
label_file176 = '/masvol/heartsmart/unet_model/baseline/data/combined_1_3_176_test_labels.npy'

perf176_df = pd.DataFrame( keys, columns=["Attribute"] )
for results_file in explist176 :
    pred_file = RESULTS_PATH + results_file + '_predictions.npy'
    perf = compute_performance_statistics(label_file176, pred_file)
    perf176_df[results_file]= perf176_df['Attribute'].map(perf)
    

In [ ]:
perf176_df

In [ ]:
RESULTS_PATH = '/masvol/heartsmart/unet_model/baseline/results/'
label_file256 = '/masvol/heartsmart/unet_model/baseline/data/combined_1_3_256_test_labels.npy'

perf256_df = pd.DataFrame( keys, columns=["Attribute"] )
for results_file in explist256 :
    pred_file = RESULTS_PATH + results_file + '_predictions.npy'
    perf = compute_performance_statistics(label_file256, pred_file)
    perf256_df[results_file]= perf256_df['Attribute'].map(perf)

In [ ]:
RESULTS_PATH = '/masvol/heartsmart/unet_model/baseline/results/'
label_file256_0 = '/masvol/heartsmart/unet_model/baseline/data/combined_1_3_0_256_test_labels.npy'

perf256_0_df = pd.DataFrame( keys, columns=["Attribute"] )
for results_file in explist256_0 :
    pred_file = RESULTS_PATH + results_file + '_predictions.npy'
    perf = compute_performance_statistics(label_file256_0, pred_file)
    perf256_0_df[results_file]= perf256_0_df['Attribute'].map(perf)

In [ ]:
perf256_df

In [ ]:
perf256_0_df

In [ ]:
x = .00001
type(x)
x

In [ ]:
file_name = '/masvol/heartsmart/unet_model/baseline/results/combined_1_3_0_256_perf_comparison.csv'
perf256_0_df.to_csv(file_name, sep='\t')

In [ ]:
file_name = '/masvol/heartsmart/unet_model/baseline/results/combined_1_3_256_perf_comparison.csv'
perf256_df.to_csv(file_name, sep='\t')

In [ ]:
#hist = '/masvol/heartsmart/unet_model/baseline/results/combined_1_3_0_256_dicecoef_learning_history.json'
#hist = '/masvol/heartsmart/unet_model/baseline/results/combined_1_3_0_256_drop_dicecoef_learning_history.json'
hist = '/masvol/heartsmart/unet_model/baseline/results/combined_1_3_0_176_learning_history.json'
hist = '/masvol/heartsmart/unet_model/baseline/results/combined_1_3_0_176_aug_drop_dice_learning_history.json'
plot_learning_history(hist)

In [ ]:
perf176_df

In [ ]:
perf176_df

In [ ]:
for results_file in explist256 :
    history_file = RESULTS_PATH + results_file + '_learning_history.json'
    print ('='*30)
    print ('='*30)
    print ("Learning history of :" + results_file)
    print ('-'*30)
    
    plot_learning_history(history_file)
    

In [ ]:
for results_file in explist176 :
    history_file = RESULTS_PATH + results_file + '_learning_history.json'
    print ('='*30)
    print ('='*30)
    print ("Learning history of :" + results_file)
    print ('-'*30)
    
    plot_learning_history(history_file)

In [ ]:
# label_file = "/masvol/heartsmart/unet_model/data/baseline/combined_1_3_256_test_labels.npy"
# pred_file = "/masvol/heartsmart/unet_model/data/baseline/combined_1_3_256_predictions.npy"
# y_true_f = np.load(label_file)
# y_pred_f = np.load(pred_file)
# p = get_performance_statistics(y_true_f, y_pred_f)
# print (p)

In [ ]:
# perf_file = "perf_values.json"
# print ("Saving Performance values", perf_file)
# with open(perf_file, 'w') as file:
#     json.dump(p, file, indent=2)
# print('-'*30)

## Analysis on training with 256x256  combined data set (ACDC + Sunnybrook)
### Below are the results of 4 tests
1. Training with combined dataset
2. Training with combined dataset + Augmentation
3. Training with combined dataset (no augmentation),  using Unet with dropout layers
4. Training with combined dataset + Augmentation,  using Unet with dropout layers

In [ ]:
history_file = '/masvol/heartsmart/unet_model/baseline/results/combined_1_3_0_256_drop_dicecoef_learning_history.json'
image_file = "/masvol/heartsmart/unet_model/baseline/data/combined_1_3_0_256_test_images.npy"
label_file = "/masvol/heartsmart/unet_model/baseline/data/combined_1_3_0_256_test_labels.npy"
pred_file = "/masvol/heartsmart/unet_model/baseline/results/combined_1_3_0_256_drop_dicecoef_predictions.npy"
plot_learning_history(history_file)
show_performance_statistics(label_file, pred_file)
find_outliers(label_file, pred_file)
display_images_labels_predictions (image_file, label_file, pred_file, num_images = 4, random_images = True)
print('-'*30)
print('-'*30)
zcontour = [21, 84, 127, 220]
display_images_labels_predictions (image_file, label_file, pred_file, image_list = zcontour)


In [ ]:
zcontour = [50, 53, 72, 73, 116, 118, 145, 188]
display_images_labels_predictions (image_file, label_file, pred_file, image_list = zcontour)

In [ ]:
history_file = "/masvol/heartsmart/unet_model/data/baseline/combined_1_3_256_learning_history.json"
image_file = "/masvol/heartsmart/unet_model/data/baseline/combined_1_3_256_test_images.npy"
label_file = "/masvol/heartsmart/unet_model/data/baseline/combined_1_3_256_test_labels.npy"
pred_file = "/masvol/heartsmart/unet_model/data/baseline/combined_1_3_256_predictions.npy"
plot_learning_history(history_file)
show_performance_statistics(label_file, pred_file)
find_outliers(label_file, pred_file)
display_images_labels_predictions (image_file, label_file, pred_file, num_images = 4, random_images = True)
print('-'*30)
print('-'*30)
zcontour = [21, 84, 127, 220]
display_images_labels_predictions (image_file, label_file, pred_file, image_list = zcontour)


In [ ]:
history_file = "/masvol/heartsmart/unet_model/data/baseline/combined_aug_1_3_256_learning_history.json"
image_file = "/masvol/heartsmart/unet_model/data/baseline/combined_1_3_256_test_images.npy"
label_file = "/masvol/heartsmart/unet_model/data/baseline/combined_1_3_256_test_labels.npy"
pred_file = "/masvol/heartsmart/unet_model/data/baseline/combined_aug_1_3_256_predictions.npy"
#plot_learning_history(history_file)
#show_performance_statistics(label_file, pred_file)
find_outliers(label_file, pred_file)
display_images_labels_predictions (image_file, label_file, pred_file, num_images = 4, random_images = True)
print('-'*30)
print('-'*30)
zcontour = [21, 84, 127, 220]
display_images_labels_predictions (image_file, label_file, pred_file, image_list = zcontour)

In [ ]:
history_file = "/masvol/heartsmart/unet_model/data/baseline/combined_drop_1_3_256_learning_history.json"
image_file = "/masvol/heartsmart/unet_model/data/baseline/combined_1_3_256_test_images.npy"
label_file = "/masvol/heartsmart/unet_model/data/baseline/combined_1_3_256_test_labels.npy"
pred_file = "/masvol/heartsmart/unet_model/data/baseline/combined_drop_1_3_256_predictions.npy"
plot_learning_history(history_file)
show_performance_statistics(label_file, pred_file)
find_outliers(label_file, pred_file)
display_images_labels_predictions (image_file, label_file, pred_file, num_images = 4, random_images = True)
print('-'*30)
print('-'*30)
zcontour = [11, 127, 165, 184, 192, 204, 217, 229]
display_images_labels_predictions (image_file, label_file, pred_file, image_list = zcontour)

In [ ]:
history_file = "/masvol/heartsmart/unet_model/data/baseline/combined_aug_drop_1_3_256_learning_history.json"
image_file = "/masvol/heartsmart/unet_model/data/baseline/combined_1_3_256_test_images.npy"
label_file = "/masvol/heartsmart/unet_model/data/baseline/combined_1_3_256_test_labels.npy"
pred_file = "/masvol/heartsmart/unet_model/data/baseline/combined_aug_drop_1_3_256_predictions.npy"
plot_learning_history(history_file)
show_performance_statistics(label_file, pred_file)
find_outliers(label_file, pred_file)
display_images_labels_predictions (image_file, label_file, pred_file, num_images = 4, random_images = True)
print('-'*30)
print('-'*30)
zcontour = [11, 21, 22, 127, 192, 204, 217, 220]
display_images_labels_predictions (image_file, label_file, pred_file, image_list = zcontour)

## Analysis on training with 176x176  combined data set (ACDC + Sunnybrook)
### Below are the results of 4 tests
1. Training with combined dataset
2. Training with combined dataset + Augmentation
3. Training with combined dataset (no augmentation),  using Unet with dropout layers
4. Training with combined dataset + Augmentation,  using Unet with dropout layers

In [ ]:
weights_file = "/masvol/heartsmart/unet_model/baseline/results/combined_1_3_176_aug_drop_dice.hdf5"
history_file = '/masvol/heartsmart/unet_model/baseline/results/combined_1_3_176_aug_drop_dice_learning_history.json'
image_file = "/masvol/heartsmart/unet_model/baseline/data/combined_1_3_176_test_images.npy"
label_file = "/masvol/heartsmart/unet_model/baseline/data/combined_1_3_176_test_labels.npy"
pred_file = "/masvol/heartsmart/unet_model/baseline/results/combined_1_3_176_aug_drop_dice_predictions.npy"
plot_learning_history(history_file)
show_performance_statistics(label_file, pred_file)
find_outliers(label_file, pred_file)
display_images_labels_predictions (image_file, label_file, pred_file, num_images = 4, random_images = True)
print('-'*30)
print('-'*30)
zcontour = [47, 68, 81, 82, 141, 155, 173, 202, 213]
display_images_labels_predictions (image_file, label_file, pred_file, image_list = zcontour)


In [ ]:
weights_file = "/masvol/heartsmart/unet_model/baseline/results/combined_1_3_176_aug_dice.hdf5"
history_file = '/masvol/heartsmart/unet_model/baseline/results/combined_1_3_176_aug_dice_learning_history.json'
image_file = "/masvol/heartsmart/unet_model/baseline/data/combined_1_3_176_test_images.npy"
label_file = "/masvol/heartsmart/unet_model/baseline/data/combined_1_3_176_test_labels.npy"
pred_file = "/masvol/heartsmart/unet_model/baseline/results/combined_1_3_176_aug_dice_predictions.npy"
plot_learning_history(history_file)
show_performance_statistics(label_file, pred_file)
find_outliers(label_file, pred_file)
display_images_labels_predictions (image_file, label_file, pred_file, num_images = 4, random_images = True)
print('-'*30)
print('-'*30)
zcontour = [47, 68, 81, 82, 141, 155, 173, 202, 213]
display_images_labels_predictions (image_file, label_file, pred_file, image_list = zcontour)


In [ ]:
zcontour = [173, 188]
display_images_labels_predictions (image_file, label_file, pred_file, image_list = zcontour)

In [ ]:
zcontour = [55, 81, 82, 141]
display_images_labels_predictions (image_file, label_file, pred_file, image_list = zcontour)

In [ ]:
weights_file = "/masvol/heartsmart/unet_model/baseline/results/combined_1_3_0_176_aug_dice.hdf5"
history_file = '/masvol/heartsmart/unet_model/baseline/results/combined_1_3_0_176_aug_dice_learning_history.json'
image_file = "/masvol/heartsmart/unet_model/baseline/data/combined_1_3_0_176_test_images.npy"
label_file = "/masvol/heartsmart/unet_model/baseline/data/combined_1_3_0_176_test_labels.npy"
pred_file = "/masvol/heartsmart/unet_model/baseline/results/combined_1_3_0_176_aug_dice_predictions.npy"
plot_learning_history(history_file)
show_performance_statistics(label_file, pred_file)
find_outliers(label_file, pred_file)
display_images_labels_predictions (image_file, label_file, pred_file, num_images = 4, random_images = True)
print('-'*30)
print('-'*30)
zcontour = [47, 68, 81, 82, 141, 155, 173, 202, 213]
display_images_labels_predictions (image_file, label_file, pred_file, image_list = zcontour)


In [ ]:
# Sample Index of labels with zero contours [8, 14, 20, 32, 39, 95, 135]
# Sample Index of predictions with zero contours [14, 20, 77, 95, 135, 169, 189]
# Sample Index where missed predictions >= 100% 6 [8, 32, 39, 77, 169, 189]
zcontour = [8, 14, 20, 32, 39, 95, 135]
display_images_labels_predictions (image_file, label_file, pred_file, image_list = zcontour)


In [ ]:
zcontour = [14, 20, 77, 95, 135, 169, 189]
display_images_labels_predictions (image_file, label_file, pred_file, image_list = zcontour)

In [ ]:
zcontour = [8, 32, 39, 77, 169, 189]
#display_images_labels_predictions (image_file, label_file, pred_file, image_list = zcontour)
display_images_predictions (image_file,  pred_file, image_list = zcontour)

###  Runs with 176x176 images + labels with zero contours (combined_1_3_0 dataset)

In [ ]:

weights_file = "/masvol/heartsmart/unet_model/baseline/results/combined_1_3_0_176_dice.hdf5"
history_file = '/masvol/heartsmart/unet_model/baseline/results/combined_1_3_0_176_dice_learning_history.json'
image_file = "/masvol/heartsmart/unet_model/baseline/data/combined_1_3_0_176_test_images.npy"
label_file = "/masvol/heartsmart/unet_model/baseline/data/combined_1_3_0_176_test_labels.npy"
pred_file = "/masvol/heartsmart/unet_model/baseline/results/combined_1_3_0_176_dice_predictions.npy"
plot_learning_history(history_file)
show_performance_statistics(label_file, pred_file)
find_outliers(label_file, pred_file)
display_images_labels_predictions (image_file, label_file, pred_file, num_images = 4, random_images = True)
print('-'*30)
print('-'*30)
zcontour = [47, 68, 81, 82, 141, 155, 173, 202, 213]
display_images_labels_predictions (image_file, label_file, pred_file, image_list = zcontour)


In [ ]:
weights_file = "/masvol/heartsmart/unet_model/baseline/results/combined_1_3_0_176_aug_drop_dice.hdf5"
history_file = '/masvol/heartsmart/unet_model/baseline/results/combined_1_3_0_176_aug_drop_dice_learning_history.json'
image_file = "/masvol/heartsmart/unet_model/baseline/data/combined_1_3_0_176_test_images.npy"
label_file = "/masvol/heartsmart/unet_model/baseline/data/combined_1_3_0_176_test_labels.npy"
pred_file = "/masvol/heartsmart/unet_model/baseline/results/combined_1_3_0_176_aug_drop_dice_predictions.npy"
plot_learning_history(history_file)
show_performance_statistics(label_file, pred_file)
find_outliers(label_file, pred_file)
display_images_labels_predictions (image_file, label_file, pred_file, num_images = 4, random_images = True)
print('-'*30)
print('-'*30)
zcontour = [47, 68, 81, 82, 141, 155, 173, 202, 213]
display_images_labels_predictions (image_file, label_file, pred_file, image_list = zcontour)


In [ ]:
zcontour = [8, 14, 20, 32, 39, 95, 135]
display_images_labels_predictions (image_file, label_file, pred_file, image_list = zcontour)

In [ ]:
#zcontour = [14, 20, 46, 65, 77, 102, 104, 154, 169, 189]
zcontour = [14, 20, 63, 77, 95, 104, 135, 169, 189, 198, 212]
display_images_labels_predictions (image_file, label_file, pred_file, image_list = zcontour)

In [ ]:
history_file = "/masvol/heartsmart/unet_model/data/baseline/combined_drop_1_3_176_learning_history.json"
image_file = "/masvol/heartsmart/unet_model/data/baseline/combined_1_3_176_test_images.npy"
label_file = "/masvol/heartsmart/unet_model/data/baseline/combined_1_3_176_test_labels.npy"
pred_file = "/masvol/heartsmart/unet_model/data/baseline/combined_drop_1_3_176_predictions.npy"
plot_learning_history(history_file)
show_performance_statistics(label_file, pred_file)
find_outliers(label_file, pred_file)
display_images_labels_predictions (image_file, label_file, pred_file, num_images = 4, random_images = True)
print('-'*30)
print('-'*30)
zcontour = [47, 68, 81, 82, 141, 155, 173, 202, 213]
display_images_labels_predictions (image_file, label_file, pred_file, image_list = zcontour)

In [ ]:
zcontour = [0, 19, 47, 55, 68, 81 ]
display_images_labels_predictions (image_file, label_file, pred_file, image_list = zcontour)

In [ ]:
#Best predictions:
bestp = [27, 112, 123]
display_images_labels_predictions (image_file, label_file, pred_file, image_list = bestp)

In [ ]:
history_file = "/masvol/heartsmart/unet_model/data/baseline/combined_aug_drop_1_3_176_learning_history.json"
image_file = "/masvol/heartsmart/unet_model/data/baseline/combined_1_3_176_test_images.npy"
label_file = "/masvol/heartsmart/unet_model/data/baseline/combined_1_3_176_test_labels.npy"
pred_file = "/masvol/heartsmart/unet_model/data/baseline/combined_aug_drop_1_3_176_predictions.npy"
plot_learning_history(history_file)
show_performance_statistics(label_file, pred_file)
find_outliers(label_file, pred_file)
display_images_labels_predictions (image_file, label_file, pred_file, num_images = 4, random_images = True)
print('-'*30)
print('-'*30)
zcontour = [47, 68, 81, 82, 141, 155, 173, 202, 213]
display_images_labels_predictions (image_file, label_file, pred_file, image_list = zcontour)

In [ ]:
## 

## Analysis of predictions on  DSB data set (256x256  images)

In [ ]:
image_file = "/masvol/heartsmart/unet_model/data/baseline/combined_1_3_256_test_images.npy"
pred_file = "/masvol/heartsmart/unet_model/data/baseline/combined_1_3_256_predictions.npy"
find_outliers_in_prediction(pred_file)
display_images_predictions (image_file, pred_file, num_images = 4, random_images = True)
print ("-"*30)
imglist = [21, 84, 127]
display_images_predictions (image_file, pred_file, image_list = imglist)


## Analysis of predictions on  DSB data set (176x176  images)

In [ ]:
image_file = "/masvol/heartsmart/unet_model/data/baseline/combined_1_3_176_test_images.npy"
pred_file = "/masvol/heartsmart/unet_model/data/baseline/combined_1_3_176_predictions.npy"
find_outliers_in_prediction(pred_file)
display_images_predictions (image_file, pred_file, num_images = 4, random_images = True)
print ("-"*30)
print ("-"*30)
imglist = [173]
display_images_predictions (image_file, pred_file, image_list = imglist)